## Аггрегация разметки датасета ruMultiAr
*Subset edition*

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
from collections import Counter

### Сбор данных разметки и фильтрация разметчиков

Данный датасет представляет из себя лишь подмножество, которое было специально сгенерировано. Это не тестовый сет датасета из мультимодального бенчмарка (инструкционный сет)! Размер выборки - 600 объектов.

In [2]:
assignments = pd.read_csv('assignments_from_pool_41562232__04-10-2023.tsv', sep='\t')

Разметчикам предлагалось решить математический пример.

Вход: 
- INPUT:formula (пример: `(2 + (-1))`).

Выход:
- OUTPUT:result (целое число, например: `1`).

In [6]:
assignments.head(1)

,INPUT:formula,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,((-6) + 9 * (-4) - (-8)),-34,NaN,NaN,NaN,https://platform.toloka.ai/task/41562232/00027...,00027a3078--6516b2ee056db464d2cd5f04,00027a3078--6516c2158879a44e701a88f7,00027a3078--6516c2158879a44e701a88e9,14098124e4a687d4023347e387bd282c,APPROVED,2023-09-29T12:24:53.371,2023-09-29T12:26:40.733,2023-09-29T12:26:40.733,0.022


Фильтруем толокеров с `accuracy < 0.5` на контрольных заданиях, чтобы не учитывать их ответы при подсчете метрик.

In [5]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    formula = row[0]

    out = row[1]
    
    gold = row[2]

    user = row[9]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  103
Bad users: 9


9 из 103 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Отделяем контроль от основы, так как контрольные задания создавались отдельно и не должны учитываться при подсчете метрик. На контрольных заданиях есть `GOLDEN:result`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:formula` содержит NaN.

In [7]:
assignments_no_control = assignments[assignments['GOLDEN:result'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:formula'].notnull()]

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [8]:
from collections import defaultdict

text_dict = defaultdict(list)

for formula, user, out in zip(
    assignments_no_control_no_null["INPUT:formula"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:result"]
    ):
    if user not in bad_users:
        text_dict[(formula)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

600


In [9]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 562, 4: 38})

Есть 38 заданий с перектрытием 4. В каждом может быть ситуация 2/2. Такие объекты пропускаем. Если есть согласие (3 голоса большинства), то оставляем, так как для перекрытия 5 порог согласия ровно такой же.

In [10]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    if len(lst) == 5:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most >= 3:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res
    elif len(lst) == 4:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most > 2:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res

In [11]:
len(preds_full)

557

In [12]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['formula']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

Достаточно много рассогласованных айтемов - 43 штуки, что объясняется спецификой задачи. Нужно написать целое число в ответ, а не выбрать кнопку, потому и вариантов бесконечность - отсюда задания, где есть большая рассогласованность. Более того, в основной пул попали примеры, в которых правильный ответ не является целым числом, что также объясняет ошибки. Такие задачи будут отфильтрованы.

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными ответами.

In [13]:
res_df = pd.read_csv('full_with_info.tsv', sep='\t')

In [14]:
res_df = res_df.rename({'INPUT:formula': 'formula', 'target': 'lb'}, axis=1)

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы

In [15]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['formula'] = res_df['formula'].apply(format_text)
preds_full_df['formula'] = preds_full_df['formula'].apply(format_text)

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [16]:
new = res_df.merge(preds_full_df, on='formula', how='left')

In [18]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

557

Ни одна строка не была утеряна.

In [20]:
new_valid.head(1)

,formula,lb_x,depth_level,length,lb_y
0,((((5 + 0) + ((-2) * 6)) + (((-8) * (-4)) + (6...,43,"[2, 2, 2]",2,43.0


### Подсчет метрики

In [21]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [22]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.9676840215439856

Это мог бы быть финальный этап, но в данные попали примеры, где ответом является не целое число, а вещественное (дробное). Учитывать такие примеры - нечестно, так как разметчики не могли ответить нецелым числом в силу специфики задания (было невозможно сохранить нецелый ответ). Уберем такие примеры.

In [23]:
new_valid['formula'].apply(lambda x: eval(x) % 1 != 0).sum()

81

В 81 из 600 объектах ответ нецелый. Убираем.

In [24]:
new_valid_filter = new_valid[new_valid['formula'].apply(lambda x: eval(x) % 1 == 0)]

In [25]:
(new_valid_filter['lb_x'] == new_valid_filter['lb_y']).mean()

1.0

`Accuracy = 1.0`

### Анализ ошибок

Нет ни одной ошибки после голосования, что снимает необходимость анализа.